In [1]:
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import win32com.client
from datetime import datetime

In [14]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [15]:
Default_Column_ZSD76=['Solic.', 'Solicitante', 'Fecha salida mercancía', 'Material',
       'Número de material', 'Cantidad entrega', 'UM', 'MARCA', 'CATEGORIA',
       'Entrega', 'ClEnt', 'Clase de entrega', 'ClVt',
       'Clase de documento de ventas', 'OrgVt', 'CDis', 'Doc.venta',
       'Prc.neto', 'Mon.', 'Destinat.', 'Destinatario de mercancías',
       'Nº pedido cliente', 'Fecha creación Pedido']

Default_Column_ZSD127=['Cliente', 'Nombre', 'Pedido Externo - OC', 'Pedido', 'Tipo',
       'Denominación', 'Fecha creación Ped.', 'Status Picking (SM)',
       'Status Factura', 'Status Transporte', 'Transporte']

In [16]:
transsacion_ZSD76='zsd76'
variant_ZSD76="JESPINOSAP"
provision_ZSd76="PROCESADOS"

In [17]:
SAP_GUI.Search_ZSD76(transsacion_ZSD76,session,provision_ZSd76,variant_ZSD76)

<COMObject <unknown>>

In [18]:
#Dowloader data

Name_ZSD76="Detalle_Entregas_ZSD76"
Ruta_ZSD76="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD76,session,Ruta_ZSD76)

In [19]:
Pedidos_Procesados=pd.read_csv(Ruta_ZSD76+Name_ZSD76+".txt",delimiter="\t",skiprows=5)
Pedidos_Procesados=Funtions.Clean_Columns(Pedidos_Procesados)
Pedidos_Procesados=Funtions.default_column(Default_Column_ZSD76,Pedidos_Procesados)

In [22]:
Transsaccion_ZSD127="zsd127"
Variant_ZSD127="ESTANDAR"
Documento_Ventas=Pedidos_Procesados['Doc.venta'].drop_duplicates()



In [24]:
SAP_GUI.Search_Pedidos_ZSD127(Transsaccion_ZSD127,Documento_Ventas,session,Variant_ZSD127)

In [10]:
#Dowloader data

Name_ZSD127="Alerta_Entregas_ZS127"
Ruta_ZSD127="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD127,session,Ruta_ZSD127)

com_error: (-2147352567, 'Ocurrió una excepción.', (619, 'SAP Frontend Server', 'The control could not be found by id.', 'C:\\Program Files (x86)\\SAP\\FrontEnd\\SAPgui\\sapfront.HLP', 393215, 0), None)

In [ ]:
Transporte_Alertas=pd.read_csv(Ruta_ZSD127+Name_ZSD127+".txt",delimiter="\t",skiprows=5)
Transporte_Alertas=Funtions.Clean_Columns(Transporte_Alertas)
#Transporte_Alertas=Funtions.default_column(Default_Column_ZSD127,Pedidos_Procesados)

In [ ]:
#Limpiar Datos
Transporte_Alertas=Transporte_Alertas.fillna(" ")

In [ ]:
"""Send email"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()


def style_df(df,column:str,value_left,value_right):
    """
    -df: Dataframe
    -column: Nombre de la columna en Str
    -value_left:Rango de valor izquierdo
    -value_right: Rango de valor derecho
    """
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \
        .highlight_between(subset=[column],color='#FF5733',left=value_left,right=value_right) \



In [ ]:
html="""
    <h2 style="text-align: center">INFORME DE PERMANENCIAS DE PEDIDOS</h2>
    <p> Por medio del presente informe se evidencia las permanencia de pedidos y el transporte asociado.</p>

    <div">{0}</div>

    <p> Anticipo sinceros agradecimientos. </p>
 """

Transporte_Alertas_Informe=style_df(Transporte_Alertas,"Transporte"," "," ")     #Style between LI and LS

send_emails(Transporte_Alertas_Informe.to_html(),emails=correos,htmlbody=html,subject="Informe de permanencia de pedidos")
